In [1]:
# Delete this cell to re-enable tracebacks
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [2]:
# JSON output syntax highlighting
from __future__ import print_function
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML

original_print = print

def json_print(inpt):
    string = str(inpt)
    if string[0] == '{':
        formatter = HtmlFormatter()
        return HTML('<style type="text/css">{}</style>{}'.format(
                    formatter.get_style_defs('.highlight'),
                    highlight(string, JsonLexer(), formatter)))
    else:
        original_print(inpt)

print = json_print

## Data Markings

### Creating Objects With Data Markings

To create an object with a (predefined) TLP marking to an object, just provide it as a keyword argument to the constructor. The TLP markings can easily be imported from python-stix2.

In [3]:
from stix2 import Indicator, TLP_AMBER

indicator = Indicator(labels=["malicious-activity"],
                      pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
                      object_marking_refs=TLP_AMBER)
print(indicator)

If you’re creating your own marking (for example, a ``Statement`` marking), first create the statement marking:

In [4]:
from stix2 import MarkingDefinition, StatementMarking

marking_definition = MarkingDefinition(                                    
    definition_type="statement",                                            
    definition=StatementMarking(statement="Copyright 2017, Example Corp")
)
print(marking_definition)

Then you can add it to an object as it’s being created (passing either full object or the the ID as a keyword argument, like with relationships).

In [5]:
indicator2 = Indicator(labels=["malicious-activity"],
                      pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
                      object_marking_refs=marking_definition)
print(indicator2)

In [6]:
indicator3 = Indicator(labels=["malicious-activity"],
                      pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
                      object_marking_refs="marking-definition--f88d31f6-486f-44da-b317-01333bde0b82")
print(indicator3)

Granular markings work in the same way, except you also need to provide a full granular-marking object (including the selector).

In [7]:
from stix2 import Malware, TLP_WHITE

malware = Malware(name="Poison Ivy",
                  labels=['remote-access-trojan'],
                  description="A ransomware related to ...",
                  granular_markings=[
                      {
                          "selectors": ["description"],
                          "marking_ref": marking_definition
                      },
                      {
                          "selectors": ["name"],
                          "marking_ref": TLP_WHITE
                      }
                  ])
print(malware)

Make sure that the selector is a field that exists and is populated on the object, otherwise this will cause an error:

In [8]:
Malware(name="Poison Ivy",
        labels=['remote-access-trojan'],
        description="A ransomware related to ...",
        granular_markings=[
            {
                "selectors": ["title"],
                "marking_ref": marking_definition
            }
        ])

InvalidSelectorError: Selector title in Malware is not valid!

### Adding Data Markings To Existing Objects

Both object markings and granular markings can also be added to STIX objects which have already been created.

**Note**: Doing so will create a new version of the object (note the updated ``modified`` time).

In [21]:
indicator4 = indicator.add_markings(marking_definition)
print(indicator4)

You can also remove specific markings from STIX objects. This will also create a new version of the object.

In [22]:
indicator5 = indicator4.remove_markings(marking_definition)
print(indicator5)

The markings on an object can be replaced with a different set of markings:

In [23]:
from stix2 import TLP_GREEN

indicator6 = indicator5.set_markings([TLP_GREEN, marking_definition])
print(indicator6)

STIX objects can also be cleared of all markings:

In [12]:
indicator7 = indicator5.clear_markings()
print(indicator7)

All of these functions can be used for granular markings by passing in a list of selectors. Note that they will create new versions of the objects.

### Evaluating Data Markings

You can get a list of the object markings on a STIX object:

In [19]:
indicator6.get_markings()

['marking-definition--34098fce-860f-48ae-8e50-ebd3cc5e41da',
 'marking-definition--030bb5c6-c5eb-4e9c-8e7a-b9aab08ded53']

To get a list of the granular markings on an object, pass the object and a list of selectors to ``get_markings``:

In [20]:
malware.get_markings('name')

['marking-definition--613f2e26-407d-48c7-9eca-b8e91df99dc9']

You can also call ``get_markings()`` as a method on the STIX object.

In [14]:
malware.get_markings('name')

['marking-definition--613f2e26-407d-48c7-9eca-b8e91df99dc9']

Finally, you may also check if an object is marked by a specific markings. Again, for granular markings, pass in the selector or list of selectors.

In [16]:
indicator.is_marked(TLP_AMBER.id)

True

In [17]:
malware.is_marked(TLP_WHITE.id, 'name')

True

In [18]:
malware.is_marked(TLP_WHITE.id, 'description')

False